In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/MIE1516 Project/' 

Mounted at /content/gdrive


In [0]:
import os
import cv2
import h5py
import numpy
import math
import skimage
from math import log10, sqrt 
from skimage.metrics import structural_similarity, peak_signal_noise_ratio

In [0]:
DATA_PATH = root_path + "/DIV2K_train_HR"
LR_DATA_PATH = root_path + "/DIV2K_train_LR_bicubic_X2/"
TEST_PATH = root_path + "/DIV2K_valid_HR/"
SAMPLE_IMAGE_TEST_PATH = root_path + "/DIV2K_valid_HR_sample/"

In [0]:
import scipy
from skimage import transform, io
import imageio
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
#Link: https://github.com/eriklindernoren/Keras-GAN
class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, batch_size=1, is_testing=False, is_random=True):
        data_type = "train" if not is_testing else "test"
        
        path = glob('%s/*' % (self.dataset_name))

        if(is_random == True):
          batch_images = np.random.choice(path, size=batch_size)
        else:
          batch_images = path[0:batch_size]

        imgs_hr = []
        imgs_lr = []
        for img_path in batch_images:
            img = self.imread(img_path)

            h, w = self.img_res
            low_h, low_w = int(h / 4), int(w / 4)

            img_hr = transform.resize(img, self.img_res)
            img_lr = transform.resize(img, (low_h, low_w))

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_hr = np.fliplr(img_hr)
                img_lr = np.fliplr(img_lr)

            imgs_hr.append(img_hr)
            imgs_lr.append(img_lr)

        imgs_hr = np.array(imgs_hr) / 127.5 - 1.
        imgs_lr = np.array(imgs_lr) / 127.5 - 1.
        print(imgs_hr.shape)
        print(imgs_lr.shape)
        return imgs_hr, imgs_lr


    def imread(self, path):
        return imageio.imread(path, pilmode='RGB').astype(np.float)


In [0]:
class ImageMetrics():
  def __init__(self):
    self.name = "Metric for Image"
  
  def print_metrics(self, reference, generated):
    self.reference = (reference)*255.0
    self.generated = (generated)*255.0
    assert self.reference.shape == self.generated.shape
    psnr_score, ssim_score = self.psnr(), self.ssim()
    print("PSNR: %.4f SSIM: %.4f" % (psnr_score, ssim_score))
    return psnr_score, ssim_score

  #Link: https://dsp.stackexchange.com/questions/38065/peak-signal-to-noise-ratio-psnr-in-python-for-an-image
  def psnr(self):
    mse = np.mean((self.reference-self.generated)**2)
    if(mse == 0):
      mse = 100
    MAX_PIXEL = 255.0
    psnr_score = 20*log10(MAX_PIXEL/sqrt(mse))
    return psnr_score

  #Link: https://scikit-image.org/docs/dev/api/skimage.metrics.html#skimage.metrics.structural_similarity
  def ssim(self):
    (ssim_score, difference) = structural_similarity(self.reference, self.generated, multichannel= True, gaussian_weights= True, full=True)
    return ssim_score

In [6]:
"""
Super-resolution of CelebA using Generative Adversarial Networks.

The dataset can be downloaded from: https://www.dropbox.com/sh/8oqt9vytwxb3s4r/AADIKlz8PR9zr6Y20qbkunrba/Img/img_align_celeba.zip?dl=0

Instrustion on running the script:
1. Download the dataset from the provided link
2. Save the folder 'img_align_celeba' to 'datasets/'
4. Run the sript using command 'python srgan.py'
"""
#Link: https://github.com/eriklindernoren/Keras-GAN
from __future__ import print_function, division
import scipy
!pip install git+https://www.github.com/keras-team/keras-contrib.git

%tensorflow_version 1.x
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam

import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import sys
#from data_loader import DataLoader
import numpy as np
import os
from collections import defaultdict
try:
    import cPickle as pickle
except ImportError:
    import pickle

import keras.backend as K

class SRCNN_PL():
    def __init__(self):
        # Input shape
        self.channels = 3
        self.lr_height = 64                 # Low resolution height
        self.lr_width = 64                  # Low resolution width
        self.lr_shape = (self.lr_height, self.lr_width, self.channels)
        self.hr_height = self.lr_height*4   # High resolution height
        self.hr_width = self.lr_width*4     # High resolution width
        self.hr_shape = (self.hr_height, self.hr_width, self.channels)

        # Number of residual blocks in the generator
        self.n_residual_blocks = 16

        optimizer = Adam(0.0002, 0.5)

        # We use a pre-trained VGG19 model to extract image features from the high resolution
        # and the generated high resolution images and minimize the mse between them
        self.vgg = self.build_vgg()
        self.vgg.trainable = False
        self.vgg.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Configure data loader
        self.save_path_name = root_path
        self.dataset_name = DATA_PATH
        self.test_dataset_name = TEST_PATH
        self.sample_test_dataset_name = SAMPLE_IMAGE_TEST_PATH
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.hr_height, self.hr_width))
        self.test_data_loader = DataLoader(dataset_name=self.test_dataset_name,
                                      img_res=(self.hr_height, self.hr_width))
        self.sample_test_data_loader = DataLoader(dataset_name=self.sample_test_dataset_name,
                                      img_res=(self.hr_height, self.hr_width))
        self.metrics = ImageMetrics()

        os.makedirs('%s/SRCNN_PL' % self.save_path_name, exist_ok=True)
        os.makedirs('%s/SRCNN_PL/checkpoints' % self.save_path_name, exist_ok=True)
        os.makedirs('%s/SRCNN_PL/sample_image_outputs' % self.save_path_name, exist_ok=True)

        self.model_gen_path = self.save_path_name + "SRCNN_PL/checkpoints/model_cnn_pl_gen_april12.h5"
        #self.model_disc_path = self.save_path_name + "SRGAN_MSE/checkpoints/model_mse_disc_april11.h5"
        self.model_combined_path = self.save_path_name + "SRCNN_PL/checkpoints/model_cnn_pl_combined_april12.h5"
        self.sample_images_path = self.save_path_name + "SRCNN_PL/sample_image_outputs"
        self.history_loss_accuracy_path = self.save_path_name + "SRCNN_PL/srcnn-pl-history.pkl"

        # Calculate output shape of D (PatchGAN)
        patch = int(self.hr_height / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        # Build and compile the discriminator
        # self.discriminator = self.build_discriminator()
        # self.discriminator.compile(loss='mse',
        #     optimizer=optimizer,
        #     metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # High res. and low res. images
        img_hr = Input(shape=self.hr_shape)
        img_lr = Input(shape=self.lr_shape)

        # Generate high res. version from low res.
        fake_hr = self.generator(img_lr)

        # Extract image features of the generated img
        fake_features = self.vgg(fake_hr)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # Discriminator determines validity of generated high res. images
        #validity = self.discriminator(fake_hr)

        self.combined = Model([img_lr, img_hr], fake_features)
        self.combined.compile(loss='mse',
                              optimizer=optimizer)


    def build_vgg(self):
        """
        Builds a pre-trained VGG19 model that outputs image features extracted at the
        third block of the model
        """
        vgg = VGG19(weights="imagenet")
        # Set outputs to outputs of last conv. layer in block 3
        # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
        vgg.outputs = [vgg.layers[9].output]

        img = Input(shape=self.hr_shape)

        # Extract image features
        img_features = vgg(img)

        return Model(img, img_features)

    def build_generator(self):

        def residual_block(layer_input, filters):
            """Residual block described in paper"""
            d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(layer_input)
            d = Activation('relu')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Add()([d, layer_input])
            return d

        def deconv2d(layer_input):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(256, kernel_size=3, strides=1, padding='same')(u)
            u = Activation('relu')(u)
            return u

        # Low resolution image input
        img_lr = Input(shape=self.lr_shape)

        # # Pre-residual block
        # c1 = Conv2D(64, kernel_size=9, strides=1, padding='same')(img_lr)
        # c1 = Activation('relu')(c1)

        # # Propogate through residual blocks
        # r = residual_block(c1, self.gf)
        # for _ in range(self.n_residual_blocks - 1):
        #     r = residual_block(r, self.gf)

        # # Post-residual block
        # c2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(r)
        # c2 = BatchNormalization(momentum=0.8)(c2)
        # c2 = Add()([c2, c1])

        # # Upsampling
        # u1 = deconv2d(c2)
        # u2 = deconv2d(u1)

        # # Generate high resolution output
        # gen_hr = Conv2D(self.channels, kernel_size=9, strides=1, padding='same', activation='tanh')(u2)
        c1 = Conv2D(128, kernel_size=9, strides=1, padding='same')(img_lr)
        c1 = Activation('relu')(c1)
        c2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(c1)
        c2 = Activation('relu')(c2)
        c3 = Conv2D(3, kernel_size=5, strides=1, padding='same')(c2)
        c3 = Activation('linear')(c3)
        u1 = deconv2d(c3)
        u2 = deconv2d(u1)#UpSampling2D(size=4)(c3)
        gen_hr = Conv2D(self.channels, kernel_size=9, strides=1, padding='same', activation='tanh')(u2)
        
        #gen_hr = Reshape(self.hr_shape)

        model_gen = Model(img_lr, gen_hr)
        print(model_gen.summary())
        return model_gen

    def build_discriminator(self):

        def d_block(layer_input, filters, strides=1, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        # Input img
        d0 = Input(shape=self.hr_shape)

        d1 = d_block(d0, self.df, bn=False)
        d2 = d_block(d1, self.df, strides=2)
        d3 = d_block(d2, self.df*2)
        d4 = d_block(d3, self.df*2, strides=2)
        d5 = d_block(d4, self.df*4)
        d6 = d_block(d5, self.df*4, strides=2)
        d7 = d_block(d6, self.df*8)
        d8 = d_block(d7, self.df*8, strides=2)

        d9 = Dense(self.df*16)(d8)
        d10 = LeakyReLU(alpha=0.2)(d9)
        validity = Dense(1, activation='sigmoid')(d10)

        model_disc = Model(d0, validity)
        print(model_disc.summary())
        return model_disc

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        if(os.path.exists(self.model_gen_path)):
          print("I am here")
          self.generator.load_weights(self.model_gen_path)

        # if(os.path.exists(self.model_disc_path)):
        #   print("I am here")
        #   self.discriminator.load_weights(self.model_disc_path)
        
        if(os.path.exists(self.model_combined_path)):
          print("I am here")
          self.combined.load_weights(self.model_combined_path)

        train_dict = defaultdict(list)
        test_dict = defaultdict(list)
        sample_imgs_hr, sample_imgs_lr = self.sample_test_data_loader.load_data(batch_size=2, is_testing=True, is_random=False)
        for epoch in range(epochs):

           
            # ------------------
            #  Train Generator
            # ------------------

            # Sample images and their conditioning counterparts
            imgs_hr, imgs_lr = self.data_loader.load_data(batch_size)

            # # The generators want the discriminators to label the generated images as real
            # valid = np.ones((batch_size,) + self.disc_patch)

            # Extract ground truth image features using pre-trained VGG19 model
            image_features = self.vgg.predict(imgs_hr)

            # Train the generators
            g_loss = self.combined.train_on_batch([imgs_lr, imgs_hr], image_features)
            print("Training Losses")
            
            print("Generator (Combined) => Training Perceptual Loss: %.4f" %(g_loss))
            train_dict["generator"].append(g_loss)

            # ------------------
            #  Test Generator
            # ------------------

            # Sample images and their conditioning counterparts
            imgs_hr, imgs_lr = self.test_data_loader.load_data(batch_size)

            # # The generators want the discriminators to label the generated images as real
            # valid = np.ones((batch_size,) + self.disc_patch)

            # Extract ground truth image features using pre-trained VGG19 model
            image_features = self.vgg.predict(imgs_hr)

            # Train the generators
            test_g_loss = self.combined.test_on_batch([imgs_lr, imgs_hr], image_features)
            print("Testing Losses")
            
            print("Generator (Combined) => Testing Perceptual Loss: %.4f" %(test_g_loss))
            test_dict["generator"].append(test_g_loss)

            elapsed_time = datetime.datetime.now() - start_time
            # Plot the progress
            print ("%d time: %s" % (epoch, elapsed_time))

            self.generator.save_weights(self.model_gen_path)
            # self.discriminator.save_weights(self.model_disc_path)
            self.combined.save_weights(self.model_combined_path)
            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch, sample_imgs_hr, sample_imgs_lr)
        
        with open(self.history_loss_accuracy_path, 'wb') as f:
          pickle.dump({'train': train_dict, 'test': test_dict}, f)

    def sample_images(self, epoch, sample_imgs_hr=None, sample_imgs_lr=None):
        #os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        if(sample_imgs_hr is None or sample_imgs_lr is None):
          imgs_hr, imgs_lr = self.test_data_loader.load_data(batch_size=2, is_testing=True)
        else:
          imgs_hr, imgs_lr = sample_imgs_hr,sample_imgs_lr

        fake_hr = self.generator.predict(imgs_lr)

        # Rescale images 0 - 1
        imgs_lr = 0.5 * imgs_lr + 0.5
        fake_hr = 0.5 * fake_hr + 0.5
        imgs_hr = 0.5 * imgs_hr + 0.5

        # Save generated images and the high resolution originals
        titles = ['LR Original','Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for row in range(r):
            psnr_score, ssim_score = self.metrics.print_metrics(reference=imgs_hr[row][2], generated=fake_hr[row][1])
            for col, image in enumerate([imgs_lr, fake_hr, imgs_hr]):
                if(row == 0):
                  if(titles[col] == 'Generated'):
                    col_title = "Generated \n PSNR: %.4f SSIM: %.4f" % (psnr_score, ssim_score)
                  else:
                    col_title = titles[col]+ "\n"
                else:
                  if(titles[col] == 'Generated'):
                    col_title = "PSNR: %.4f SSIM: %.4f" % (psnr_score, ssim_score)
                  else:
                    col_title = "\n"
                axs[row, col].imshow(image[row])
                axs[row, col].set_title(col_title)
                axs[row, col].axis('off')
            cnt += 1
        fig.savefig("%s/%d.png" % (self.sample_images_path, epoch))
        plt.close()

        # Save low resolution images for comparison
        if((epoch == 0 and (sample_imgs_hr is not None or sample_imgs_lr is not None)) or (sample_imgs_hr is None or sample_imgs_lr is None)):
          for i in range(r):
              fig = plt.figure()
              plt.imshow(imgs_lr[i])
              fig.savefig('%s/%d_lowres%d.png' % (self.sample_images_path, epoch, i))
              plt.close()

if __name__ == '__main__':
    tf.reset_default_graph()
    try:
        del srcnn_model
    except:
        pass
    K.clear_session()

    
    graph = tf.get_default_graph()

    with graph.as_default():
      srcnn_model = SRCNN_PL()
      srcnn_model.train(epochs=201, batch_size=10, sample_interval=50)


  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-gpfq1fug
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-gpfq1fug
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=d586a7b59c784a8432def1c1c88e43dbc7c59f0f3524e95f70bea51a9de1ba10
  Stored in directory: /tmp/pip-ephem-wheel-cache-yydy63z5/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
TensorFlow 1.x selected.


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       31232     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        73792     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 3)         4803      
___________________________________________

/usr/local/lib/python3.6/dist-packages/skimage/metrics/_structural_similarity.py:108: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  im2[..., ch], **args)


PSNR: 9.3476 SSIM: -0.0002
PSNR: 15.0519 SSIM: -0.0001
(10, 256, 256, 3)
(10, 64, 64, 3)
Training Losses
Generator (Combined) => Training Perceptual Loss: 77.0260
(10, 256, 256, 3)
(10, 64, 64, 3)
Testing Losses
Generator (Combined) => Testing Perceptual Loss: 54.0539
1 time: 0:01:52.509873
(10, 256, 256, 3)
(10, 64, 64, 3)
Training Losses
Generator (Combined) => Training Perceptual Loss: 62.5606
(10, 256, 256, 3)
(10, 64, 64, 3)
Testing Losses
Generator (Combined) => Testing Perceptual Loss: 70.8620
2 time: 0:02:40.307239
(10, 256, 256, 3)
(10, 64, 64, 3)
Training Losses
Generator (Combined) => Training Perceptual Loss: 68.2677
(10, 256, 256, 3)
(10, 64, 64, 3)
Testing Losses
Generator (Combined) => Testing Perceptual Loss: 86.8990
3 time: 0:03:26.986998
(10, 256, 256, 3)
(10, 64, 64, 3)
Training Losses
Generator (Combined) => Training Perceptual Loss: 66.8597
(10, 256, 256, 3)
(10, 64, 64, 3)
Testing Losses
Generator (Combined) => Testing Perceptual Loss: 70.1090
4 time: 0:04:14.4228